# Temel Bileşen Regresyonu : Principal Component Regression (PCR)

**Çoklu Doğrusal Bağıntı** kısaca, değişkenlerin birbirlerini açıklaması durumudur. Yani değişkenler arasında negatif veya pozitif güçlü kolerasyonlar vardır ve bu durum **yanlılığa** neden olabilmektedir. Bu problemi önlemek, birbirleriyle tamamen bağımsız değişkenler ortaya çıkarmak ve bunları modellemek için  **Temel Bileşen Regresyonu (PCR)** kullanılır.

**Temel Bileşen Regresyonu**, Temel Bileşen Analizi ile seçilen bileşenler ile bir model kurmak üzerine çalışmaktadır. Temel Bileşen Analizi P adet değişkenin içerdiği bilgiyi ondan daha az sayıda değişken ile bir miktar veri kaybını göz önünde bulundurarak maksimum şekilde ifade etmek için kullanılır. **Böylelikle çok boyutluluk ve değişkenlerin birbirleriyle olan korelasyonu ortadan kalkar.** Yüksek korelasyon değişkenlerin benzer tahminsel şeyleri barındırdığını gösterir.

PCR’ın SciKit-Learn’de direkt uygulaması olmadığı için bunu kendimiz yapmalıyız. PCR uygulaması için;
1. PCA yapılacak.
2. Regresyon modeline oturtulacak.

In [3]:
import pandas as pd
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263 entries, 1 to 321
Data columns (total 20 columns):
AtBat        263 non-null int64
Hits         263 non-null int64
HmRun        263 non-null int64
Runs         263 non-null int64
RBI          263 non-null int64
Walks        263 non-null int64
Years        263 non-null int64
CAtBat       263 non-null int64
CHits        263 non-null int64
CHmRun       263 non-null int64
CRuns        263 non-null int64
CRBI         263 non-null int64
CWalks       263 non-null int64
League       263 non-null object
Division     263 non-null object
PutOuts      263 non-null int64
Assists      263 non-null int64
Errors       263 non-null int64
Salary       263 non-null float64
NewLeague    263 non-null object
dtypes: float64(1), int64(16), object(3)
memory usage: 43.1+ KB


In [5]:
df.isnull().sum()

AtBat        0
Hits         0
HmRun        0
Runs         0
RBI          0
Walks        0
Years        0
CAtBat       0
CHits        0
CHmRun       0
CRuns        0
CRBI         0
CWalks       0
League       0
Division     0
PutOuts      0
Assists      0
Errors       0
Salary       0
NewLeague    0
dtype: int64

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
AtBat,263.0,403.642586,147.307209,19.0,282.5,413.0,526.0,687.0
Hits,263.0,107.828897,45.125326,1.0,71.5,103.0,141.5,238.0
HmRun,263.0,11.619772,8.757108,0.0,5.0,9.0,18.0,40.0
Runs,263.0,54.745247,25.539816,0.0,33.5,52.0,73.0,130.0
RBI,263.0,51.486692,25.882714,0.0,30.0,47.0,71.0,121.0
Walks,263.0,41.114068,21.718056,0.0,23.0,37.0,57.0,105.0
Years,263.0,7.311787,4.793616,1.0,4.0,6.0,10.0,24.0
CAtBat,263.0,2657.543726,2286.582929,19.0,842.5,1931.0,3890.5,14053.0
CHits,263.0,722.186312,648.199644,4.0,212.0,516.0,1054.0,4256.0
CHmRun,263.0,69.239544,82.197581,0.0,15.0,40.0,92.5,548.0


In [7]:
dms = pd.get_dummies(df[['League','Division','NewLeague']])
dms.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0


In [8]:
y = df["Salary"]

In [9]:
X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis = 1).astype("float64")

In [10]:
X_.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0


In [11]:
X =  pd.concat([X_, dms[['League_N','Division_W','NewLeague_N']]], axis = 1)
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


In [12]:
#Kategorik Değişkenleri Sürekli Değişkene Çevirme Alternatif Yol
#lbe = LabelEncoder()  #LabelEncoder kullanarak kategorik değişkenleri 1 veya 0'a dönüştürüyoruz.
#df["League"] = lbe.fit_transform(df["League"])
#df["Division"] = lbe.fit_transform(df["Division"])
#df["NewLeague"] = lbe.fit_transform(df["NewLeague"])

In [13]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

print("X_train", X_train.shape)

print("y_train",y_train.shape)

print("X_test",X_test.shape)

print("y_test",y_test.shape)

training = df.copy()

print("training", training.shape)

X_train (197, 19)
y_train (197,)
X_test (66, 19)
y_test (66,)
training (263, 20)


**Verinin ölçeklendirilmesi ve bileşenlerin oluşturulması**

PCR’ın pythonda direkt uygulaması olmadığı için sıradaki işlemimiz PCA nesnesi oluşturmak olacak. Ve daha sonra X_train verimizi scale (ölçeklendirme) işlemini yapacağız.

In [14]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
pca = PCA() #Normalde PCA n_components yani bileşen sayısı parametresini alır. 
            #Eğer parametreyi vermezsek bütün bileşenleri kullanır.

In [15]:
X_reduced_train = pca.fit_transform(scale(X_train)) #scale -> ölçeklendirme

C:\Users\Hakan-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.


In [21]:
import numpy as np
np.cumsum(np.round(pca.explained_variance_ratio_, decimals = 4)*100)[0:10] #kümülatif olarak açıklanan varyans

array([38.18, 59.88, 70.88, 78.88, 84.18, 88.45, 92.05, 94.86, 96.34,
       97.28])

In [24]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
pcr_model = lm.fit(X_reduced_train, y_train)

In [25]:
pcr_model.intercept_

543.4834416243655

In [26]:
pcr_model.coef_

array([ 111.13977427,  -29.34209502,   26.29799759,  -38.47549852,
        -56.9200785 ,   54.44779423,   40.77493384,  -23.72746012,
          9.31198164,   13.02031672,   45.58357748,   31.97791627,
         18.93930958, -115.60940171,   24.00382778,  415.70806202,
       -449.51779543,  563.07375399,  302.53718462])

# Tahmin 

In [27]:
y_pred = pcr_model.predict(X_reduced_train)

In [28]:
y_pred[0:5]

array([377.44484744, 802.19452124, 495.60987745, 112.53177731,
       426.21613066])

In [34]:
from sklearn.metrics import mean_squared_error, r2_score
np.sqrt(mean_squared_error(y_train, y_pred))

289.32928255649756

In [35]:
df["Salary"].mean()

535.9258821292775

In [36]:
r2_score(y_train, y_pred)

0.577007525041018

In [37]:
pca2 = PCA()

In [38]:
X_reduced_test = pca2.fit_transform(scale(X_test))

C:\Users\Hakan-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.


In [40]:
y_pred = pcr_model.predict(X_reduced_test) #eğitim seti üzerinden, kurmuş olduğumuz modeli kullanarak test setinin x değerlerini
#modele yerleştirip y'nin tahmin edilen değerlerini elde etmiş olduk

In [41]:
np.sqrt(mean_squared_error(y_test, y_pred))

405.1575364149967